## RCS Python Closures and Generators

In [68]:
## Closures - binding variables from outer function in the inner function
## Technically - function gets stored with its enviroment(bound variables)
### Can also think of preserving certain state

In [1]:
# remember this function?
def add_factory(x):
    def add(y):
        return y + x 
    return add # upon return free variable x gets bound in the add function


In [2]:
add5 = add_factory(5)
# 5 is bound inside add5 now,
add5(10)

15

In [4]:
type(add5.__closure__)

tuple

In [5]:
[x for x in add5.__closure__]

[<cell at 0x04E79470: int object at 0x649F2900>]

In [6]:
len(add5.__closure__)

1

In [8]:
int(add5.__closure__[0])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'cell'

In [9]:
dir(add5.__closure__[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'cell_contents']

In [10]:
add5.__closure__[0].cell_contents

5

In [ ]:
## Voila!! We get what we expected to get!

In [ ]:
## Remember __closure__ is a tuple so we do not get to mutate this!

In [ ]:
So how about more values stored?

In [12]:
def add2_fact(x, y):
    return lambda z: z+x+y

In [13]:
a10n20 = add2_fact(10,20)

In [14]:
a10n20(50)

80

In [15]:
len(a10n20.__closure__)

2

In [16]:
[x.cell_contents for x in a10n20.__closure__]

[10, 20]

In [ ]:
One last closure example:

In [47]:
def outer(x):
    a = 20
    def inner(y):
        print(f'x:{x}')
        print(f'a:{a}')
        print(f'y:{y}')
        ## x += 15 We can't change the argument that was bound from outside argument
        ## a += 15 We can't change the a that was bound from outside function
        return a+x+y
    return inner

In [48]:
axy = outer(10)

In [49]:
axy(5)

x:10
a:20
y:5


35

In [31]:
axy(5)

35

In [32]:
[x.cell_contents for x in axy.__closure__]

[20, 10]

## What if we want rebind(assign new values) to variables coming from outer scope?
### In languages like Javascript you can do it, so Python should be able to, right?
### Solution: Python3 nonlocal modifier inside inner function 

In [57]:
# https://docs.python.org/3/reference/simple_stmts.html#the-nonlocal-statement
# 7.13. The nonlocal statement

# The nonlocal statement causes the listed identifiers to refer to previously bound variables in the nearest enclosing scope excluding globals. 
# This is important because the default behavior for binding is to search the local namespace first. The statement allows encapsulated code to rebind variables outside of the local scope besides the global (module) scope.

# Names listed in a nonlocal statement, unlike those listed in a global statement, must refer to pre-existing bindings in an enclosing scope (the scope in which a new binding should be created cannot be determined unambiguously).

# Names listed in a nonlocal statement must not collide with pre-existing bindings in the local scope.

In [52]:
def makeCounter():
    count = 0
    def f():
        nonlocal count
        count +=1
        return count
    return f


In [53]:
a = makeCounter()

In [54]:
a()

1

In [56]:
a()

3

In [55]:
a()

2

In [70]:
def makeAdjCounter(x):
    count = 0
    def f():
        nonlocal count  # without nonlocal we could reference count but couldn't modify it!
        count += x
        return count
    return f

In [61]:
b = makeAdjCounter(2)
c = makeAdjCounter(3)

In [63]:
print(b(),b(),b())

4 6 8


In [66]:
print(c(),c(),c(),c())

12 15 18 21


In [69]:
[x.cell_contents for x in c.__closure__]

[21, 3]

In [58]:
# Result count is hidden from us, but by calling function we can modify its value.


In [78]:
## Another older way was to create some structure(List, Class, Dictionary) inside outer function whose members could be modified by innner


In [76]:
def makeAdjList():
    holder=[1] # old method not recommended anymore!
    def f():
        holder[0] +=1
        return holder[0]
    return f

In [75]:
d = makeAdjList()


In [77]:
print(d(),d(),d())

2 3 4


### Most Python answer is to use generators for persisting some sort of iterable state

## What the heck is a Generator ?

###  A Python generator is a function which returns a generator iterator (just an object we can iterate over) by calling yield

* KEY POINT: generator functions use **yield** instead of **return**
* in Python 3 we use next(generatorName) to obtain next value

In [95]:
def makeNextGen(current):
    while True: ##This means our generator will never run out of values...
        current += 1
        yield current
    

In [90]:
numGen = makeNextGen(30)

In [91]:
for i in range(5):
    print(next(numGen))  # This is for Python 3.x ,  in Python 2.x it was numGen.next()

31
32
33
34
35


In [93]:
## We can do even better and make an adjustable increment

In [96]:
def makeNextGenInc(current, inc):
    while True: 
        current += inc
        yield current


In [97]:
numGen10 = makeNextGenInc(200, 10)

In [100]:
[next(numGen10) for x in range(10)]

[210, 220, 230, 240, 250, 260, 270, 280, 290, 300]

In [101]:
## Now the above is Pythonic approach to the problem!

In [116]:
### Then there is a generator expression 
## The whole point is have a lazy evaluation (ie no need to have everything at once in memory)


In [165]:
gen = (i+10 for i in range(10))


In [166]:
list(gen)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [154]:
type(gen)

generator

In [167]:
gen = (i+10 for i in range(10))

In [159]:
for g in gen:
    print(g)

10
11
12
13
14
15
16
17
18
19


In [163]:
for g in gen:
    print(g)

In [164]:
# You see what is going on?!

In [157]:
gen_exp = (x ** 2 for x in range(10) if x % 2 == 0)
type(gen_exp)

generator

In [158]:
for x in gen_exp:
    print(x)

0
4
16
36
64


In [149]:
glist = list(gen)
glist

[]

In [113]:
[next(gen) for x in range(5)]

[461, 462, 463, 464, 465]

In [150]:
yes_expr = ('yes' for _ in range(10))

In [152]:
list(yes_expr)

['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']

In [114]:
## Challenge how to make an infinite generator with a generator expression?

In [124]:
import itertools

In [132]:
genX = (i*5 for i in itertools.count(start=0, step=1))


In [133]:
[next(genX) for x in range(10)]

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

In [140]:
import random
genY = (i*10+random.randrange(10) for i in itertools.count(start=0, step=1))

In [141]:
[next(genY) for x in range(10)]

[3, 18, 21, 33, 46, 56, 60, 74, 82, 90]

In [168]:
## Be very careful with infinite generators, calling list on infinite generator not recommended!

In [129]:
## Of course we should generally have a pretty good idea of maximum number of generations needed

### Difference between Python's Generators and Iterators
* iterators is more general, covers all generators

From Official Docs: Python’s generators provide a convenient way to implement the iterator protocol. If a container object’s __iter__() method is implemented as a generator, it will automatically return an iterator object (technically, a generator object) supplying the __iter__() and next()

## A Generator is an Iterator
### Specifically, generator is a subtype of iterator.

Conceptually:
Iterators are about various ways to loop over data, generators generate the data on the fly

## Homework
### Write a generator to yield cubes (forever!)
### Write a generator to yield Fibonacci numbers(first 1000)

* Generator Functions ok to use here